<a href="https://colab.research.google.com/github/popescuaaa/playground/blob/master/Simple_GAN_Improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>